In [2]:
from math import *
import numpy as np

k = 0.01720209895
cAU = 173.144643267
eps = radians(23.4374)

def HMStoDegOrRad(h, m, s, option):
    deg = (h * 15) + (m * (1/60) * 15) + (s * (1/3600) * 15)
    if (option):
        return deg
    else:
        return deg * (math.pi/180)
        
def DMStoDeg(d, am, asec):
    return ((float(d)) + (copysign(float(am), float(d)) * (1/60)) + (copysign(float(asec), float(d)) * (1/3600)))

def FindAngle(s, c):
    return (atan2(s, c) + 2*np.pi) % (2*np.pi)

def RAdecimalToHMS(val):
    h = trunc(val / 15)
    val = (abs(val) / 15) - abs(h)
    m = trunc(val / (1/60))
    val = val - (m * (1/60))
    s = val / (1/3600)
    s = round(s, 0)
    if (s == 60):
        s = 0
        m += 1
    print ("RA:", h, ":", m, ":", s)

def DECdecimalToDMS(dec):
    d = trunc(dec)
    dec = abs(dec) - abs(d)
    m = trunc(dec / (1/60))
    dec -= (m * (1/60))
    s = dec / (1/3600)
    s = round(s, 0)
    if (s == 60):
        s = 0
        m += 1
    print ("Dec:", d, ":", m, ":", s)
    
def GetMagnitude(v):
    return sqrt(v[0]**2 + v[1]**2 + v[2]**2)

def AltAzToRADec(alt, az, lst):
    lst = HMStoDegOrRad(lst[0], lst[1], lst[2], False)
    lat = 40 * np.pi/180
    az = az * np.pi/180
    alt = alt * np.pi/180
    
    dec = np.arcsin(np.sin(lat) * np.sin(alt) + np.cos(lat) * np.cos(alt) * np.cos(az))

    HA = FindAngle((np.sin(2*np.pi - az) * np.cos(alt) / np.cos(dec)), (np.sin(alt) - np.sin(lat) * np.sin(dec)) / (np.cos(lat) * np.cos(dec)))
    RA = lst - HA
    
    RA = RA * 180/np.pi / 15
    if RA < 0:
        RA += 24
    
    dec = dec * 180/np.pi
    
    return RA, dec
    
def DotVectors(v1, v2):
    return v1[0]*v2[0]+v1[1]*v2[1]+v1[2]*v2[2]
    
def CrossVectors(v1, v2):
    return [v1[1]*v2[2] - v1[2]*v2[1], -(v1[0]*v2[2] - v2[0]*v1[2]), v1[0]*v2[1] - v2[0]*v1[1]]
    
def TripleProduct(v1, v2, v3):
    return DotVectors(CrossVectors(v2, v3), v1)
    
def Rot3DVector(v, a, b):
    rotMatrix = np.array([[np.cos(a), np.sin(a), 0], [-np.sin(a)*np.cos(b), np.cos(b)*np.cos(a), np.sin(b)], [np.sin(a)*np.sin(b), -np.cos(a)*np.sin(b), np.cos(b)]])
    result = np.matmul(rotMatrix, v)
    return result

def InputCSV(path):
    data = np.loadtxt(path, dtype=float, delimiter=',')
    return np.array(data)

# Runs fine
def CalculateAngularMomentum(v1, v2):
    return [round((v1[1]*v2[2] - v1[2]*v2[1]), 6), round(-(v1[0]*v2[2] - v2[0]*v1[2]), 6), round((v1[0]*v2[1] - v2[0]*v1[1]), 6)]
    
#CalculateAngularMomentum('/home/vumat/Desktop/Python_Code/VuInput.csv')

path = '/home/vumat/Desktop/Python_Code/VuInput.csv'

def PrintOrbitalElements(value, expected, error):
    print('Calculated Value:', value)
    print('Expected Value:', expected)
    print('Error(%):', error)

def CalculateOrbitalElements(path):
    ad = InputCSV(path)
    conversionRate = 58.13244
    r = np.array(ad[0:3]) 
    v = np.array(ad[3:6]) * conversionRate
    h = CalculateAngularMomentum(r, v)
    t = ad[6]
    expected = ad[7:len(ad)]
    
    # Calculate orbital elements
    a = 1 / ((2/GetMagnitude(r)) - (GetMagnitude(v) ** 2))
    e = np.sqrt(1 - ((GetMagnitude(h)) ** 2 / a))
    i = np.arccos(h[2] / GetMagnitude(h))
    omega = FindAngle((h[0] / (GetMagnitude(h) * np.sin(i))), (-h[1] / (GetMagnitude(h) * np.sin(i))))
    
    U = FindAngle((r[2] / (GetMagnitude(r) * np.sin(i))), (r[0] * np.cos(omega) + r[1] * np.sin(omega)) / GetMagnitude(r))
    nu = FindAngle((a * (1 - e ** 2) / (GetMagnitude(h) * e) * DotVectors(r, v) / GetMagnitude(r)), (1 / e) * ((a * (1 - e ** 2)) / GetMagnitude(r) - 1))
    w = U - nu
    if w < 0:
        w += 2 * np.pi
    
    E = FindAngle(GetMagnitude(r) * np.sin(nu) / (a * np.sqrt(1 - e ** 2)), (e + np.cos(nu)) / (1 + e * np.cos(nu)))
    M = E - e * np.sin(E)
    T = t - M / (k / a ** 1.5)
    
    i = degrees(i)
    omega = degrees(omega)
    w = degrees(w)
    E = degrees(E)
    M = degrees(M)
    
    # Calculate percent errors
    da = abs((a - expected[9]) / expected[9]) * 100
    de = abs((e - expected[0]) / expected[0]) * 100
    di = abs((i - expected[2]) / expected[2]) * 100
    domega = abs((omega - expected[3]) / expected[3]) * 100
    dw = abs((w - expected[4]) / expected[4]) * 100
    dM = abs((M - expected[7]) / expected[7]) * 100
    dT = abs((T - expected[5]) / expected[5]) * 100
    
    print('Semi-major axis(AU):\n')
    PrintOrbitalElements(a, expected[9], da)
    print('\nEccentricity:\n')
    PrintOrbitalElements(e, expected[0], de)
    print('\nInclination(degrees):\n')
    PrintOrbitalElements(i, expected[2], di)
    print('\nOmega(degrees):\n')
    PrintOrbitalElements(omega, expected[3], domega)
    print('\nAngle of perihelion(degrees):\n')
    PrintOrbitalElements(w, expected[4], dw)
    print('\nMean anomaly(degrees):\n')
    PrintOrbitalElements(M, expected[7], dM)
    print('\nTime of perihelion passage(days):\n')
    PrintOrbitalElements(T, expected[5], dT)
    
    
CalculateOrbitalElements(path)

IndexError: index 2 is out of bounds for axis 0 with size 1